In [1]:
import utils as ut
import scipy as sc
import numpy as np
from tqdm.notebook import tqdm
import scipy.sparse as sparse
from joblib import Parallel, delayed


def hadamard_matmal(r,X,B,cols,ind):
    idx = cols[ind[r]:ind[r+1]]
    return (B[idx][:,idx]).sum(0)

def WDLAE_cg(pc, X, XtX_lam, weight, nonzeros, path, users, items):
    ind = X.indptr
    B = np.zeros((items,items))
    XWX = (X.T.dot(weight*X)).toarray() # 30 seconds
    temp = Parallel(n_jobs=36)(delayed(hadamard_matmal)(user,X,B,nonzeros[1],ind) for user in range(users)) # 2 minutes
    WXXB = (weight - 1) * sparse.csr_matrix((np.concatenate(temp), (nonzeros[0], nonzeros[1])))
    XtXB = np.array(X.T.dot(WXXB) + XtX_lam.dot(B)) # 1.5 minutes...
    r = XWX - XtXB
    z = pc.dot(r) # 1.5 minutes 
    p = z.copy()
    for _ in tqdm(range(5)):
        temp = Parallel(n_jobs=36)(delayed(hadamard_matmal)(user,X,p,nonzeros[1],ind) for user in range(users))
        WXXp = (weight - 1) * sparse.csr_matrix((np.concatenate(temp), (nonzeros[0], nonzeros[1])))
        XtXp = np.array(X.T.dot(WXXp) + XtX_lam.dot(p))
        rtz = np.inner(r.flatten('F'),z.flatten('F'))  
        alpha = rtz / max(1e-32,np.inner(p.flatten('F'),XtXp.flatten('F')))
        B += alpha * p
        r -= alpha * XtXp
        rnorm = np.linalg.norm(r.flatten('F'))
        if rnorm <= 1e-8:
            ut.evaluate(B,path)
            break
        if _ % 5 == 4:
            ut.evaluate(B,path)
        z = pc.dot(r)
        beta = np.inner(r.flatten('F'),z.flatten('F')) / rtz
        p = z + beta * p
    

#paths = ['/efs/users/hsteck/public/datasets/msd_data/pro_sg', '/efs/users/hsteck/public/datasets/movielens20mio/pro_sg']
path = '/efs/users/hsteck/public/datasets/movielens20mio/pro_sg'

Ps = [0.01,0.1,0.5,0.9,0.99]
weights = [1,2,5,10,20]
X = ut.load_train_data(path)
users,items = X.shape[0],X.shape[1]
XtX = (X.T@X).toarray()
nonzeros = X.nonzero()

In [ ]:
for p in Ps:
    XtX_lam = XtX.copy()
    XtX_lam[np.diag_indices(items)] *= (1 + p/(1-p))
    pc = np.linalg.inv(XtX_lam)
    for weight in weights:
        print('running with (p,weight): ', p,' , ', weight)
        WDLAE_cg(pc, X, XtX_lam, weight, nonzeros, path, users, items)

running with (p,weight):  0.01  ,  1


  0%|          | 0/5 [00:00<?, ?it/s]

Validation Metrics
Val NDCG@100=0.38376 (0.00221)
Val Recall@20=0.35643 (0.00271)
Val Recall@50=0.47666 (0.00294)
Test Metrics
Test NDCG@100=0.37786 (0.00219)
Test Recall@20=0.35506 (0.00270)
Test Recall@50=0.47530 (0.00296)
running with (p,weight):  0.01  ,  2


  0%|          | 0/5 [00:00<?, ?it/s]

Val NDCG@100=0.37851 (0.00220)
Val Recall@20=0.35216 (0.00272)
Val Recall@50=0.47564 (0.00297)
Test Metrics
Test NDCG@100=0.37215 (0.00218)
Test Recall@20=0.35062 (0.00271)
Test Recall@50=0.47405 (0.00298)
running with (p,weight):  0.01  ,  5


...

Validation Metrics
Val NDCG@100=0.35527 (0.00204)
Val Recall@20=0.31900 (0.00258)
Val Recall@50=0.43834 (0.00282)
Test Metrics
Test NDCG@100=0.34773 (0.00202)
Test Recall@20=0.31462 (0.00258)
Test Recall@50=0.43350 (0.00284)
running with (p,weight):  0.99  ,  10


  0%|          | 0/5 [00:00<?, ?it/s]

Validation Metrics
Val NDCG@100=0.37238 (0.00206)
Val Recall@20=0.33648 (0.00260)
Val Recall@50=0.45764 (0.00281)
Test Metrics
Test NDCG@100=0.36496 (0.00203)
Test Recall@20=0.33310 (0.00260)
Test Recall@50=0.45406 (0.00284)
running with (p,weight):  0.99  ,  20


  0%|          | 0/5 [00:00<?, ?it/s]

Validation Metrics
Val NDCG@100=0.38606 (0.00207)
Val Recall@20=0.35133 (0.00262)
Val Recall@50=0.47410 (0.00281)
Test Metrics
